In [5]:
from tpu_utility_1 import *
!apt-get install libturbojpeg
!pip install jpeg4py albumentations pillow

from IPython.display import clear_output
clear_output()

In [6]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import os
import albumentations
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm
import jpeg4py as jpeg

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [7]:
import torch 
import torch.nn as nn
import torch.utils.data as D
import torchvision.transforms as T 
from torchvision import models

diseases = ['healthy', 'rust', 'scab', 'frog_eye_leaf_spot', 'powdery_mildew', 'complex']
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

ModuleNotFoundError: No module named 'torch'

In [9]:
labels = df.labels.str.split()
occ = np.zeros((len(labels),len(diseases)))
count = 0
for i in diseases:
    occ[:,count] = [i in list for list in labels]
    count+=1

dis_df = pd.DataFrame(occ.astype(np.float32), columns=diseases)
torch_df = pd.concat([df["image"], dis_df], axis=1)
torch_df.head()

NameError: name 'df' is not defined

In [10]:
skf = StratifiedKFold(n_splits=5)

tdf_np = df.to_numpy()
X = tdf_np[:, 0]
y = tdf_np[:, 1]

for train_index, test_index in skf.split(X, y): pass

torch_df.to_numpy()[train_index]  # we use the last train index and test index here as demo. 

NameError: name 'df' is not defined

In [12]:
class AppleDataset(D.Dataset):
    def __init__(self, parent_path, df=None, shuffle=False, seed=None, transforms=None, split=None):
        """
        parent_path: (pathlib.Path) Parent path of images. 
        df: (pandas.DataFrame) y-labels dataframe. 
        shuffle: (Boolean) Shuffle dataset? Default: False. 
        seed: (integer) seed. Default: None. 
        transforms: (Pytorch Transforms) Transformation to images, default: None. 
        split: (python list) Splits index for train test split. 
        """
        self.df_np = df.to_numpy()
        self.parent_path = parent_path
        self.seed = seed
        self.transforms = transforms

        if type(split) != type(None): self.df_np = self.df_np[split]

    def __len__(self):
        return len(self.df_np)

    def __getitem__(self, idx):
        item = self.df_np[idx]
        item_path = self.parent_path / item[0]  # contains the image filenames. 
        image = jpeg.JPEG(item_path).decode()  # read jpeg file (slightly quicker than pillow-simd)

        if self.transforms is not None:
            image = self.transforms(**{"image": image})
            image = image["image"]

        image = torch.from_numpy(image).permute(2, 0, 1)  # HWC -> CHW format. 
        target = torch.from_numpy(item[1:].astype(np.float32))

        return image, target

NameError: name 'D' is not defined

In [13]:
size = 224  # final image size. 

train_transform = albumentations.Compose([
    albumentations.RandomResizedCrop(height=size, width=size, always_apply=True, scale=(0.75, 1.0)),
    albumentations.Flip(),
    albumentations.Rotate(limit=10, p=0.75),
    albumentations.RandomBrightnessContrast(p=0.75),
    albumentations.Normalize(always_apply=True, p=1.0)
])

val_transform = albumentations.Compose([
    albumentations.RandomResizedCrop(height=size, width=size, always_apply=True, scale=(0.9, 1.0)),
    albumentations.Normalize(always_apply=True, p=1.0)
])

Note the above transform isn't the best transform to apply to my dataset, but it serves as a demonstration that this is what you do if you are to create a cache dataset. Result isn't important here, just the technique which you could apply to your work. 

In [14]:
FLAGS = {
    "data_dir": Path("../input/plant-pathology-2021-fgvc8/train_images"),
    "bs": 32,  # batch size "per core". Will be multipled by xm.xrt_world_size() later. 
    "num_workers": os.cpu_count(),
    "lr": 0.001,  # can be tweaked. Will be multiplied by xm.xrt_world_size() later. 
    "momentum": 0.9,  # if using SGD, which we aren't
    "num_epochs": 10,
    "num_cores": 8 if os.environ.get("TPU_NAME", None) else 1,  # how many TPU cores to use.
    "log_steps": 20,  # not used here. Required if you want to print every (your number) steps. 
    "metrics_debug": False,
    "seed": 1397,
    "save_path": "/kaggle/working/model.pth"
}

In [15]:
import torch_xla.utils.cached_dataset as xcd

ModuleNotFoundError: No module named 'torch_xla'

# `get_dataset()`
This is how you create `get_dataset()`. The return required is `train_ds` and `val_ds`. 

In [16]:
def get_dataset():
    parent_path = FLAGS["data_dir"]
    
    train_ds = AppleDataset(parent_path, torch_df, shuffle=True, seed=FLAGS["seed"], 
                            transforms=train_transform, split=train_index)
    val_ds = AppleDataset(parent_path, torch_df, shuffle=False, seed=FLAGS["seed"],
                          transforms=val_transform, split=test_index)

    return train_ds, val_ds

We can then create the cached dataset. We pass in `compress` to create a `.tar.gz` file instead of just open folders (which is the default). This is so that we can create kaggle dataset. If you would like zip file, it is currently not supported, but you can change the program yourself by zipping it with shutil or bash code. 

In [18]:
from tpu_cache_ds_utils import cache_dataset

cache_dataset(get_dataset=get_dataset, compress=True)

ModuleNotFoundError: No module named 'torch_xla'

# Pass in train_ds and val_ds instead

If instead of get_dataset you get train_ds and val_ds. 

In [19]:
train_ds, val_ds = get_dataset()

cache_dataset(train_ds, val_ds, compress=True)

NameError: name 'AppleDataset' is not defined